In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
np.random.seed(42)
tf.random.set_seed(42)
from scikeras.wrappers import KerasClassifier

2025-10-16 00:30:27.209180: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-16 00:30:27.209802: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 00:30:27.289775: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
census = pd.read_csv("USCensusTraining.csv")
census.columns = [col.strip() for col in census.columns]
census['income'] = census['income'].map({'>50K.':1,'<=50K.':0}) 

In [3]:
# Replace '?' with dominant category of each column (Deal with null values)
census.replace('?', np.nan, inplace=True)
print(len(census))

# Drop the 1 observation with Holand-Netherlands native country, as this is not covered in the test data
census = census[census['native-country'] != 'Holand-Netherlands']
print(len(census))
categorical_cols = ['workclass', 'education', 'marital-status','occupation','relationship','race','sex','native-country']
for col in categorical_cols:
    dominant_value = census[col].mode()[0]
    census[col] = census[col].fillna(dominant_value)

25000
24999


In [4]:
census.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24999 entries, 0 to 24999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             24999 non-null  int64 
 1   workclass       24999 non-null  object
 2   demogweight     24999 non-null  int64 
 3   education       24999 non-null  object
 4   education-num   24999 non-null  int64 
 5   marital-status  24999 non-null  object
 6   occupation      24999 non-null  object
 7   relationship    24999 non-null  object
 8   race            24999 non-null  object
 9   sex             24999 non-null  object
 10  capital-gain    24999 non-null  int64 
 11  capital-loss    24999 non-null  int64 
 12  hours-per-week  24999 non-null  int64 
 13  native-country  24999 non-null  object
 14  income          24999 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 3.1+ MB


In [5]:
# OneHotEncoder for categorical var 
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded = encoder.fit_transform(census[categorical_cols])
encoded_cols = encoder.get_feature_names_out(categorical_cols)

census_encoded = pd.concat([
    pd.DataFrame(encoded, columns=encoded_cols),
    census.drop(columns=categorical_cols).reset_index(drop=True)
], axis=1)

In [6]:
X = census_encoded.drop(columns=["income"])
y = census_encoded["income"]
np.bincount(y)

array([19015,  5984])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42) 
scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
def build_model(optimizer="RMSprop", lr=1e-3, units=32, act="sigmoid"):
    model = Sequential(name="ANN_Tunable")
    model.add(Input(shape=(X_train_scaled.shape[1],), name="input_features")) 
    model.add(Dense(units, activation=act, kernel_initializer="glorot_uniform", name="hidden"))
    model.add(Dense(1, activation="sigmoid", name="output"))
    
    if isinstance(optimizer, str):
        opt_name = optimizer.lower()
        if opt_name == "rmsprop":
            opt = tf.keras.optimizers.RMSprop(learning_rate=lr)
        elif opt_name in ("sgd", "sgd_m"):
            opt = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
        else:
            opt = tf.keras.optimizers.get(optimizer)
    else:
        opt = optimizer
        
    model.compile(optimizer=opt,loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [9]:
clf = KerasClassifier(model=build_model, epochs=40, batch_size=32, verbose=2, random_state=42)
param_grid = {
    "model__units": [32, 64],
    "model__optimizer": ["RMSprop", "sgd"],
    "model__lr": [1e-2, 1e-3],
    "model__act": ["relu","sigmoid"],
    "epochs": [20],
    "batch_size": [32]   
}

In [10]:
random_search = RandomizedSearchCV(
    clf,
    param_distributions=param_grid,
    n_iter=20,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)

In [11]:
random_search.fit(X_train_scaled, y_train)

/opt/conda/lib/python3.12/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
2025-10-16 00:30:36.639591: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-10-16 00:30:36.731280: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-10-16 00:30:36.774336: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-10-16 00:30:36.802541: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:varian

Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20


2025-10-16 00:30:36.892616: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
2025-10-16 00:30:36.941935: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

417/417 - 2s - 4ms/step - accuracy: 0.8282 - loss: 0.3672
Epoch 2/20
417/417 - 2s - 4ms/step - accuracy: 0.8315 - loss: 0.3634
Epoch 2/20
417/417 - 2s - 4ms/step - accuracy: 0.8289 - loss: 0.3626
Epoch 2/20
417/417 - 2s - 4ms/step - accuracy: 0.8301 - loss: 0.3624
Epoch 2/20
417/417 - 1s - 2ms/step - accuracy: 0.8403 - loss: 0.3397
Epoch 3/20
417/417 - 1s - 2ms/step - accuracy: 0.8452 - loss: 0.3424
Epoch 3/20
417/417 - 1s - 2ms/step - accuracy: 0.8399 - loss: 0.3403
Epoch 3/20
417/417 - 1s - 3ms/step - accuracy: 0.8424 - loss: 0.3440
Epoch 3/20
417/417 - 1s - 2ms/step - accuracy: 0.8456 - loss: 0.3317
Epoch 4/20
417/417 - 1s - 2ms/step - accuracy: 0.8458 - loss: 0.3370
Epoch 4/20
417/417 - 1s - 3ms/step - accuracy: 0.8492 - loss: 0.3358
Epoch 4/20
417/417 - 1s - 3ms/step - accuracy: 0.8451 - loss: 0.3317
Epoch 4/20
417/417 - 1s - 2ms/step - accuracy: 0.8487 - loss: 0.3268
Epoch 5/20
417/417 - 1s - 2ms/step - accuracy: 0.8519 - loss: 0.3311
Epoch 5/20
417/417 - 1s - 3ms/step - accuracy

2025-10-16 00:30:59.953325: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


...

2025-10-16 00:35:44.051700: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


209/209 - 0s - 1ms/step
Epoch 1/20


2025-10-16 00:35:44.450326: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-10-16 00:35:44.582486: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


625/625 - 1s - 2ms/step - accuracy: 0.8197 - loss: 0.3811
Epoch 2/20
625/625 - 1s - 1ms/step - accuracy: 0.8389 - loss: 0.3464
Epoch 3/20
625/625 - 1s - 1ms/step - accuracy: 0.8424 - loss: 0.3388
Epoch 4/20
625/625 - 1s - 2ms/step - accuracy: 0.8448 - loss: 0.3340
Epoch 5/20
625/625 - 1s - 1ms/step - accuracy: 0.8475 - loss: 0.3305
Epoch 6/20
625/625 - 1s - 1ms/step - accuracy: 0.8495 - loss: 0.3277
Epoch 7/20
625/625 - 1s - 1ms/step - accuracy: 0.8510 - loss: 0.3255
Epoch 8/20
625/625 - 1s - 2ms/step - accuracy: 0.8516 - loss: 0.3237
Epoch 9/20
625/625 - 1s - 992us/step - accuracy: 0.8528 - loss: 0.3222
Epoch 10/20
625/625 - 1s - 958us/step - accuracy: 0.8538 - loss: 0.3208
Epoch 11/20
625/625 - 1s - 937us/step - accuracy: 0.8536 - loss: 0.3196
Epoch 12/20
625/625 - 1s - 1ms/step - accuracy: 0.8542 - loss: 0.3186
Epoch 13/20
625/625 - 1s - 962us/step - accuracy: 0.8544 - loss: 0.3177
Epoch 14/20
625/625 - 1s - 974us/step - accuracy: 0.8547 - loss: 0.3167
Epoch 15/20
625/625 - 1s - 936

,estimator,KerasClassifi..._weight=None )
,param_distributions,"{'batch_size': [32], 'epochs': [20], 'model__act': ['relu', 'sigmoid'], 'model__lr': [0.01, 0.001], ...}"
,n_iter,20
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [12]:
print("Best params:", random_search.best_params_)
print("Best CV accuracy", random_search.best_score_)

Best params: {'model__units': 64, 'model__optimizer': 'RMSprop', 'model__lr': 0.001, 'model__act': 'relu', 'epochs': 20, 'batch_size': 32}
Best CV accuracy 0.8504925413795318


In [13]:
# Checking Over-fitting
model = build_model(optimizer="RMSprop", lr=0.001, units=64, act="relu")
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2,verbose=2)

y_train_prob = model.predict(X_train_scaled)
y_test_prob = model.predict(X_test_scaled)

y_train_pred = [round(y[0]) for y in y_train_prob]
y_test_pred = [round(y[0]) for y in y_test_prob]

Epoch 1/20


2025-10-16 00:35:59.656277: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


500/500 - 1s - 2ms/step - accuracy: 0.8156 - loss: 0.3921 - val_accuracy: 0.8305 - val_loss: 0.3594
Epoch 2/20
500/500 - 1s - 1ms/step - accuracy: 0.8394 - loss: 0.3464 - val_accuracy: 0.8332 - val_loss: 0.3503
Epoch 3/20
500/500 - 1s - 1ms/step - accuracy: 0.8442 - loss: 0.3391 - val_accuracy: 0.8370 - val_loss: 0.3454
Epoch 4/20
500/500 - 1s - 1ms/step - accuracy: 0.8458 - loss: 0.3346 - val_accuracy: 0.8400 - val_loss: 0.3421
Epoch 5/20
500/500 - 1s - 1ms/step - accuracy: 0.8482 - loss: 0.3313 - val_accuracy: 0.8428 - val_loss: 0.3398
Epoch 6/20
500/500 - 1s - 1ms/step - accuracy: 0.8492 - loss: 0.3287 - val_accuracy: 0.8432 - val_loss: 0.3380
Epoch 7/20
500/500 - 1s - 1ms/step - accuracy: 0.8501 - loss: 0.3266 - val_accuracy: 0.8443 - val_loss: 0.3365
Epoch 8/20
500/500 - 1s - 1ms/step - accuracy: 0.8514 - loss: 0.3249 - val_accuracy: 0.8443 - val_loss: 0.3354
Epoch 9/20
500/500 - 1s - 1ms/step - accuracy: 0.8522 - loss: 0.3233 - val_accuracy: 0.8438 - val_loss: 0.3344
Epoch 10/20


2025-10-16 00:36:12.977890: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step


In [14]:
# Training data
print("=== Training Data ===")
print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_train_pred))
print("Classification Report:\n", classification_report(y_train, y_train_pred))

# Test data
print("=== Test Data ===")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("Classification Report:\n", classification_report(y_test, y_test_pred))

=== Training Data ===
Accuracy: 0.8571928596429822
Confusion Matrix:
 [[14093  1119]
 [ 1737  3050]]
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91     15212
           1       0.73      0.64      0.68      4787

    accuracy                           0.86     19999
   macro avg       0.81      0.78      0.79     19999
weighted avg       0.85      0.86      0.85     19999

=== Test Data ===
Accuracy: 0.85
Confusion Matrix:
 [[3512  291]
 [ 459  738]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.92      0.90      3803
           1       0.72      0.62      0.66      1197

    accuracy                           0.85      5000
   macro avg       0.80      0.77      0.78      5000
weighted avg       0.84      0.85      0.85      5000



In [15]:
model.summary()

Model: "ANN_Tunable"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 64)             │         6,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,572 (53.02 KB)

 Trainable params: 6,785 (26.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,787 (26.52 KB)

In [16]:
# Run best model with the entire data
# Fit on entire training data set
scaler = preprocessing.MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
history = model.fit(X_scaled, y, epochs=40, batch_size=32, validation_split=0.2, verbose=2)

census_final = pd.read_csv("USCensusTest.csv")
census_final.columns = [col.strip() for col in census_final.columns]

census_final.replace('?', np.nan, inplace=True)
for col in categorical_cols:
    dominant_value_final = census_final[col].mode()[0]
    census_final[col] = census_final[col].fillna(dominant_value_final)

encoded_final = encoder.fit_transform(census_final[categorical_cols])
encoded_cols_final = encoder.get_feature_names_out(categorical_cols)
census_final_encoded = pd.concat([
    pd.DataFrame(encoded_final, columns=encoded_cols_final),
    census_final.drop(columns=categorical_cols).reset_index(drop=True)
    ], axis=1)

X_final_scaled = scaler.transform(census_final_encoded)

predictions = model.predict(X_final_scaled)
rounded = [round(x[0]) for x in predictions]

Epoch 1/40
625/625 - 1s - 1ms/step - accuracy: 0.8531 - loss: 0.3188 - val_accuracy: 0.8606 - val_loss: 0.3107
Epoch 2/40
625/625 - 1s - 1ms/step - accuracy: 0.8535 - loss: 0.3171 - val_accuracy: 0.8594 - val_loss: 0.3116
Epoch 3/40
625/625 - 1s - 1ms/step - accuracy: 0.8542 - loss: 0.3158 - val_accuracy: 0.8596 - val_loss: 0.3122
Epoch 4/40
625/625 - 1s - 1ms/step - accuracy: 0.8540 - loss: 0.3147 - val_accuracy: 0.8588 - val_loss: 0.3127
Epoch 5/40
625/625 - 1s - 1ms/step - accuracy: 0.8546 - loss: 0.3137 - val_accuracy: 0.8592 - val_loss: 0.3130
Epoch 6/40
625/625 - 1s - 2ms/step - accuracy: 0.8548 - loss: 0.3127 - val_accuracy: 0.8592 - val_loss: 0.3131
Epoch 7/40
625/625 - 1s - 1ms/step - accuracy: 0.8554 - loss: 0.3119 - val_accuracy: 0.8604 - val_loss: 0.3133
Epoch 8/40
625/625 - 1s - 1ms/step - accuracy: 0.8556 - loss: 0.3112 - val_accuracy: 0.8586 - val_loss: 0.3135
Epoch 9/40
625/625 - 1s - 1ms/step - accuracy: 0.8565 - loss: 0.3105 - val_accuracy: 0.8594 - val_loss: 0.3137
E

2025-10-16 00:36:45.027852: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


In [17]:
with open("USCensusResult.txt", "w") as f:
    for pred in rounded:
        f.write(f"{pred}\n")